# Homework: Vector Search


### Imports

In [25]:
from sentence_transformers import SentenceTransformer
import requests 
import numpy as np
from tqdm.notebook import trange, tqdm

## Q1. Getting the embeddings model

First we obtain the model that will be used to generate the embeddings.

In [3]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Then, a user question is generated and its embedding is computed.

In [8]:
user_question = "I just discovered the course. Can I still join it?"
user_question_emb = embedding_model.encode(user_question)
print(user_question_emb[0])

0.078222655


**Answer**: 0.07

## Q2. Creating the embeddings

First, we will prepare the documents list (with ids already included).

In [10]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [12]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

We will only need the subset of questions regarding *machine-learning-zoomcamp*. Let's filter.

In [14]:
documents_filtered = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']

In [16]:
print(len(documents_filtered))

375


Now it is time to generate the embedding for both question and answer fields of all the documents.

In [26]:
embeddings = []

for doc in tqdm(documents_filtered):
    qa_text = f"{doc['question']} {doc['text']}"
    embeddings.append(embedding_model.encode(qa_text))

X = np.array(embeddings)

  0%|          | 0/375 [00:00<?, ?it/s]

In [24]:
print(X.shape)

(375, 768)


**Answer**: (375, 768)

## Q3. Search